In [1]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
# from utils import *
from collections import Counter
from tqdm import tqdm
import time
import geopandas as gpd
import gdown # for downloading files from google drive
import shutil
# ignore warnings
import warnings
import sys
warnings.filterwarnings("ignore")

In [14]:
analysis_results = pd.read_pickle('analysis_results.pkl')
analysis_results

,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality
0,Brightkite Checkins Graph,6493,292973,90.242723,0.713999,8.778480,3.013369,0.000534
1,Gowalla Checkins Graph,3073,62790,40.865604,0.548372,8.030410,3.508031,0.001277
2,Foursquare Checkins Graph,2324,246702,212.30809,0.65273,7.751045,2.186112,0.000938
3,Brightkite Friendship Graph,5420,14690,5.420664,0.218571,8.597851,5.231807,0.000664
4,(Filtered) Gowalla Friendship Graph,2294,5548,4.836966,0.234293,7.738052,5.396488,0.001331
5,Foursquare Friendship Graph,1397,5323,7.620616,0.183485,7.242082,6.45841,0.001531


In [15]:
analysis_results['omega-coefficient'] = np.nan
analysis_results

,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality,omega-coefficient
0,Brightkite Checkins Graph,6493,292973,90.242723,0.713999,8.778480,3.013369,0.000534,NaN
1,Gowalla Checkins Graph,3073,62790,40.865604,0.548372,8.030410,3.508031,0.001277,NaN
2,Foursquare Checkins Graph,2324,246702,212.30809,0.65273,7.751045,2.186112,0.000938,NaN
3,Brightkite Friendship Graph,5420,14690,5.420664,0.218571,8.597851,5.231807,0.000664,NaN
4,(Filtered) Gowalla Friendship Graph,2294,5548,4.836966,0.234293,7.738052,5.396488,0.001331,NaN
5,Foursquare Friendship Graph,1397,5323,7.620616,0.183485,7.242082,6.45841,0.001531,NaN


In [16]:
# rename (Filtered) Gowalla Friendship Graph in Gowalla Friendship Graph
analysis_results.loc[analysis_results['Graph'] == 'Filtered Gowalla Friendship Graph', 'Graph'] = 'Gowalla Friendship Graph'
analysis_results

,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality,omega-coefficient
0,Brightkite Checkins Graph,6493,292973,90.242723,0.713999,8.778480,3.013369,0.000534,NaN
1,Gowalla Checkins Graph,3073,62790,40.865604,0.548372,8.030410,3.508031,0.001277,NaN
2,Foursquare Checkins Graph,2324,246702,212.30809,0.65273,7.751045,2.186112,0.000938,NaN
3,Brightkite Friendship Graph,5420,14690,5.420664,0.218571,8.597851,5.231807,0.000664,NaN
4,(Filtered) Gowalla Friendship Graph,2294,5548,4.836966,0.234293,7.738052,5.396488,0.001331,NaN
5,Foursquare Friendship Graph,1397,5323,7.620616,0.183485,7.242082,6.45841,0.001531,NaN


In [18]:
# Foursquare Checkins Graph : -0.056
# Gowalla Checkins Graph : -0.24
# Brightkite Checkins Graph : -0.18

# add omega-coefficient to the respective graphs
analysis_results.loc[analysis_results['Graph'] == 'Foursquare Checkins Graph', 'omega-coefficient'] = -0.056
analysis_results.loc[analysis_results['Graph'] == 'Gowalla Checkins Graph', 'omega-coefficient'] = -0.24
analysis_results.loc[analysis_results['Graph'] == 'Brightkite Checkins Graph', 'omega-coefficient'] = -0.18
analysis_results

,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality,omega-coefficient
0,Brightkite Checkins Graph,6493,292973,90.242723,0.713999,8.778480,3.013369,0.000534,-0.180
1,Gowalla Checkins Graph,3073,62790,40.865604,0.548372,8.030410,3.508031,0.001277,-0.240
2,Foursquare Checkins Graph,2324,246702,212.30809,0.65273,7.751045,2.186112,0.000938,-0.056
3,Brightkite Friendship Graph,5420,14690,5.420664,0.218571,8.597851,5.231807,0.000664,NaN
4,(Filtered) Gowalla Friendship Graph,2294,5548,4.836966,0.234293,7.738052,5.396488,0.001331,NaN
5,Foursquare Friendship Graph,1397,5323,7.620616,0.183485,7.242082,6.45841,0.001531,NaN


In [25]:
# rename (Filtered) Gowalla Friendship Graph in Gowalla Friendship Graph
analysis_results.loc[analysis_results['Graph'] == '(Filtered) Gowalla Friendship Graph', 'Graph'] = 'Gowalla Friendship Graph'

In [26]:
# FourSquare Friendship Graph : -0.17
# Gowalla Friendship Graph : -0.25
# Brightkite Friendship Graph : -0.20

# add omega-coefficient to the respective graphs
analysis_results.loc[analysis_results['Graph'] == 'Foursquare Friendship Graph', 'omega-coefficient'] = -0.17
analysis_results.loc[analysis_results['Graph'] == 'Gowalla Friendship Graph', 'omega-coefficient'] = -0.25
analysis_results.loc[analysis_results['Graph'] == 'Brightkite Friendship Graph', 'omega-coefficient'] = -0.20

In [27]:
analysis_results


,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality,omega-coefficient
0,Brightkite Checkins Graph,6493,292973,90.242723,0.713999,8.778480,3.013369,0.000534,-0.180
1,Gowalla Checkins Graph,3073,62790,40.865604,0.548372,8.030410,3.508031,0.001277,-0.240
2,Foursquare Checkins Graph,2324,246702,212.30809,0.65273,7.751045,2.186112,0.000938,-0.056
3,Brightkite Friendship Graph,5420,14690,5.420664,0.218571,8.597851,5.231807,0.000664,-0.200
4,Gowalla Friendship Graph,2294,5548,4.836966,0.234293,7.738052,5.396488,0.001331,-0.250
5,Foursquare Friendship Graph,1397,5323,7.620616,0.183485,7.242082,6.45841,0.001531,-0.170


In [ ]:
# save the results into a pickle file
analysis_results.to_pickle('analysis_results.pkl')